In [ ]:
#default_exp random

In [ ]:
#export 
from kesscore.imports import *

In [ ]:
#export
def _rangechoice(nprs,len,size,replace,cum_weights,p):
    size,pyrs = ifnone(size,1),random.Random(nprs.random())
    if replace:
        assert p is None,'Should use cum_weights'
        if cum_weights is None: return nprs.randint(len,size=size)
        else:                 return pyrs.choices(range(len),k=size,cum_weights=cum_weights) 
                                     # ^ here can also provide weight=p but performencewise not.
    else:
        assert cum_weights is None,'Should use p'
        if p is None: return pyrs.sample(range(len),size)
        else:         return nprs.choice(np.arange(len),size=size,replace=False,p=p)
    
def _choice(nprs,a,size=None,replace=True,cum_weights=None,p=None):
    if isinstance(a,(dict, range)): a=list(a)
    if isinstance(a,(tuple,list )): a=L(a)
    assert isinstance(a, (L,np.ndarray,torch.Tensor))
    idx = _rangechoice(nprs, len=len(a),size=size,replace=replace,cum_weights=cum_weights,p=p)
    if size == None: idx=idx[0] # instead or returning something like a[[1]] will rerturn a[1]
    return a[idx]

In [ ]:
def testc(sz,rp,cw=None,p=None): return _choice(np.random.RandomState(10),[1,2,3,5,6,7],sz,rp,cw,p)
test_eq_type(testc(None,False),2)
test_eq_type(testc(None,True),6)
test_eq_type(testc(1,True),L([6]))
test_eq_type(testc(10,True), L([6, 1, 2, 5, 6, 2, 7, 1, 7, 2]))
test_eq_type(testc(6,False), L([2, 6, 3, 7, 5, 1]))
test_fail(lambda:testc(7,False),contains='Sample larger than population or is negative')

a = np.random.RandomState(10).randn(10,3)
def testa(sz,rp,cw=None,p=None): return _choice(np.random.RandomState(10),a,sz,rp,cw,p)
test_eq_type(testa(None,True), a[4])
test_eq_type(testa(1,True), a[[4]])
testa(3,True,cw=[0.1] + [0.0]*9)
p = (np.arange(10)+1)**3 
p = p.astype(np.float64)  / p.sum()
pcum = p.cumsum()
test_eq_type(testa(3, True), a[[4,0,1]])
test_eq_type(testa(3, True, cw=pcum), a[[-1,-1,-1]])
test_eq_type(testa(3, False, p=p), a[[3,8,9]])

In [ ]:
#export
@docs
class RandomState:
    def __init__(self, seed=None): self.rs=np.random.RandomState(seed=seed)
    def choice(self,a,size=None,cum_weights=None): return _choice(self.rs,a,size=size,replace=True, cum_weights=cum_weights)
    def sample(self,a,size=None,cum_weights=None): return _choice(self.rs,a,size=size,replace=False,cum_weights=cum_weights)
    def randn(self,size=None):return self.rs.randn(*size)
    def rand(self,size=None): return self.rs.random(size=size)
    def bool(self,size=None,p=0.5): return self.rand(size) < p
    def int(self,low,high=None,size=None,dtype=int): return self.rs.randint(low=low,high=high,size=size,dtype=dtype)
    _docs = dict(
        cls_doc = "The class docstring",
        choice  = 'Get sample with replacements',
        sample  = 'Get sample w.o  replacements',
        randn   = 'Return floats in [0, 1)',
        rand    = 'Return floats from normal distribution N(0,1)',
        bool    = 'Return True with probability p',
        int     = 'Return int(ndarray) with in [low, high) ([0, low))',
    )

In [ ]:
test_eq(RandomState(10).bool(size=10), [False,True,False,False,True,True,True,False,True,True])
test_eq_type(RandomState(10).bool(), False)
test_eq_type(RandomState(10).bool(p=0.9), True)
test_eq_type(RandomState(10).bool(p=0.1), False)
test_eq(RandomState(10).choice([1,2,3]), 1)
test_eq(RandomState(10).choice({1:2, 3:4}, 3), [1, 3, 1])
test_eq(RandomState(10).sample({1:2, 3:4}, 2), [1,3])
test_fail(lambda: RandomState(10).sample([1,2], 3), contains='Sample larger than population or is negative')
test_eq(np.round(RandomState(10).rand(3), 3), [0.771, 0.021, 0.634])
test_eq(RandomState(10).int(9,size=4), [4, 0, 1, 0])

test_eq(RandomState(10).int(9,size=1000).max(), 8)
test_eq(RandomState(10).int(9,size=1000).min(), 0)

test_eq(RandomState(10).int(1, 9,size=1000).max(), 8)
test_eq(RandomState(10).int(1, 9,size=1000).min(), 1)

In [ ]:
from nbdev.sync import notebook2script

In [ ]:
notebook2script()

Converted 00_functional.ipynb.
Converted 01_images.ipynb.
Converted 02_fastdl.ipynb.
Converted 03_datadownload.ipynb.
Converted 04_random.ipynb.
Converted index.ipynb.
